---

<p><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Oracle_logo.svg/2560px-Oracle_logo.svg.png" width="200" align = "left"></p>

# **<h1 align ="middle"><b> Oracle CloudWorld - Las Vegas</b></h1>**

### **<h1 align ="middle"><b> Use case 1. Person Detection in Video</b></h1>**
### **<h1 align ="middle"><b> Use case 2. Offensive Language Detection in Video</b></h1>**

---

## **Prerequisites to run notebook**
- **Note 1**: Perform all steps in the "1_prep_ocw_las_vegas_v1.ipynb" file
- **Note 2**: Pull Github Repository, the repo contains:
    - Three notebooks:
    - A folder named 'job_artifacts', inside the 'job_artifacts' folder:
        - main.py
        - sub_packages with 5 .py files
        - An empty 'wallet' folder
- **Note 3**: A Dynamic Group for OCI Data Science has been created and OCI Data Science policies are in order. Documentation on policies and dynamic group can be found [here](https://docs.oracle.com/en-us/iaas/data-science/using/policies.htm)
- **Note 4**: A correct config file with private key
- **Note 5**: Only use .jpg images as profile images.
- **Note 6**: YouTube video language should be in English (only for Video analysis)

----

# **Steps in notebook**
---
### !!! Please follow the steps carefully !!!
----


# **Step 1**

### **Config file and Private Key**
- 1. Go to Oracle Cloud homepage. Click on the person icon (top-right) and select 'User settings'.
- 2. On the left, click on 'API Keys'. Click on 'Add API Key'
- 3. Select the 'Generate API Key Pair' option. Download the Private Key.
- 4. Click on 'Add Key'. Copy the configuration (starting with [DEFAULT]) into a .txt file and save as 'config'. (without .txt as extension)
- 5. In the config file, change the last line to "key_file=./private_key.pem". Make sure to rename your private key to "private_key.pem".
- 6. Upload your 'config' file and your 'private_key.pem' file into the 'job_artifacts' folder
----
# **Step 2**

### **Autonomous Database Wallet**
- 1. Go to OCI Homepage, click on 'Oracle Database' in the menu bar and select 'Autonomous Database'
- 2. Navigate to the correct compartment and select your autonomous database (on which APEX runs)
- 3. Click on the database name to open the overview page of your autonomous database. Click on 'Database Connection'
- 4. Click on Download Wallet and enter your password
- 5. In the notebook session, upload the entire wallet in the "job_artifact/wallet" folder.
- 6. Do not change the name of the wallet. Do not unzip the wallet.
----

# **Step 3**

### **Parameters in main.py**
- 1. In "job_artifacts" folder, you'll see a file "main.py".
- 2. Open main.py. In the first few lines, ONLY CHANGE:
    - password --> This is password of your user in the Autonomous Database. Example: password = "thisismypassword123"
    - wallet_name --> This is the name of the wallet/database, without the 'Wallet_' and without '.zip'. Example: wallet_name = "DB202112101358"  
    - compartment_id_input --> This is the Compartment OCID you are working in. Example: compartment_id_input = "ocid1.compartment.oc1..aaaaaaaae3n6r6hrjipbap2hojicrsvkzatrtlwvsyrjd7wjnw4za3m75q"
- 3. After changing the parameters, save the file (CTRL+S) and close the file.
----
# **Step 4**

### **Log Group OCID**
- 1. In the OCI Data Science - Job, we need a Log Group. In this Log Group, individual Logs will be added on the individual Jobs
- 2. Create a Log Group by following these [steps](https://docs.oracle.com/en-us/iaas/Content/Logging/Task/create-logging-log-group.htm)
- 3. Copy the Log Group OCID and paste in the next cell. This ocid will be used in .with_log_group_id() during the creating of the Job. Example: with_log_group_id("ocid1.loggroup.oc1.eu-frankfurt-1.amaaaaaangx44ts2shyv7azqbxf4q")

----


##### Run the below cell and enter your Log Group OCID in the below cell. For example: ***ocid1.loggroup.oc1.eu-frankfurt-1.amaaaaaangencdyajxalcuggjaug57r3ugare7olsk44ts2shyv7azqbxf4q***

In [ ]:
## Run this cell by click on the 'play' button. An empty box appears. Add your Log Group OCID in the empty box and hit enter.
log_group_ocid = input()
print("Your input for Log Group OCID is " + log_group_ocid)

# **Step 5**

### **Custom Conda**
- 1. In the '1_prep_ocw_v1.ipynb' notebook, you created a custom conda environment. That environment is published on Object Storage.
- 2. In the top right corner in the notebook session, make sure to select your custom conda environment, like "Python [YOUR_CUSTOM_CONDA_NAME]]
- 3. Get the full location of the custom conda in the bucket. Example: "oci://[BUCKET_NAME]@[NAMESPACE]/conda_environments/cpu/ocw_las_vegas_v1/1.0/ocw_las_vegas_v1_0"
- 4. Paste the full location, including "oci://" in next cell. Example: ***oci://[BUCKET_NAME]@[NAMESPACE]/conda_environments/cpu/ocw_las_vegas_v1/1.0/ocw_las_vegas_v1_v1_0***


In [ ]:
# Please run this cell
# example of full Custom Conda path = oci://West_BP@frqap2zhtzbe/conda_environments/cpu/ocw_las_vegas/1.0/ocw_las_vegas_v1_0
custom_conda_path = input()
print("Your input for custom conda is " + custom_conda_path)

---

# **Step 6**

### **Run the notebook below**
#### Run each cell below by clicking on the 'Play' button or step into the cell and hit shift+enter
- 1. Import libraries
- 2. Create an OCI Data Science - Job
- 3. Start the OCI Data Science - Job

---

# **| 1. Import libraries**

In [ ]:
#imports
from ads.common.oci_logging import OCILogGroup, OCILog
from ads.jobs import Job, DataScienceJob, PythonRuntime
from datetime import datetime, timedelta
from ads import set_auth

#authentication
from ads import set_auth
set_auth(auth='resource_principal')

---

# **| 2. Create an OCI Data Science - Job**

In [ ]:
## the below script creates an OCI Data Science - Job

job = (
    Job(name="ocw_my_first_job")
    .with_infrastructure(
        DataScienceJob()
        # Configure logging for getting the job run outputs.
        .with_log_group_id(log_group_ocid)                                                                                                 ###### Add here your Log Group OCID
        .with_shape_name("VM.Standard2.8")                                                                                                 ###### Optional. Change shape to your liking.
        #.with_shape_config_details(memory_in_gbs=16, ocpus=5)
        .with_block_storage_size(50)
    )
    .with_runtime(
        PythonRuntime()
        .with_custom_conda(custom_conda_path)                                                                                               ###### Add here your Custom Conda Location
        .with_source("/home/datascience/ocw_las_vegas/job_artifacts/") 
        .with_entrypoint("main.py")
        .with_working_dir("job_artifacts")
        .with_python_path("sub_packages")
        .with_environment_variable(TYPE_OF_ANALYSIS="both",   
                                  YOUTUBE_URL = "no_url_provided",
                                  MAIN_BUCKET_NAME = 'no_bucket_name', 
                                  NAMESPACE_NAME = 'no_name_space_name', 
                                  SCHEMA_NAME = 'no_schema_name'  
                                  )         
    )
)

job.create()

---

# **| 3. Start the OCI Data Science - Job**

##### Before you start a Job, make sure to add or change the below values.

### **Input Features are:**
- **TYPE_OF_ANALYSIS** --> Use "audio_only", "video_only", or "both"
- **YOUTUBE_URL** --> The full YouTube URL
- **MAIN_BUCKET_NAME** --> This is the main bucket used. All sub buckets (based on SCHEMA_NAME) will be added to the main bucket
- **NAMESPACE_NAME** --> Namespace of Object Storage
- **SCHEMA_NAME** --> This is the user name / schema name of the database. Will also be used to create subbuckets in the main bucket.

#### Example YouTube videos of Jimmy Carr:
- https://www.youtube.com/shorts/BsDFzV-Hpsw
- https://www.youtube.com/embed/epU3ipIBSqA?autoplay=1&mute=1    

---

## **Enter your inputs here - Add or change your inputs and run the cell:**

Please use 'audio_only' in this Notebook. For 'video_only' or 'both', you need to upload a profile image (.jpg) in the Object Storage Bucket. This has not been done yet.

In [ ]:
type_of_analysis = "audio_only"
youtube_url = "https://www.youtube.com/shorts/tjYJu8E1m4k"
main_bucket_name = "West_BP"
namespace_name = "frqap2zhtzbe"
schema_name = "ocw"

## **Start Job Run**

In [ ]:
job_run_env = job.run(
    name="Job Run - Passing dynamic values",
    env_var={'TYPE_OF_ANALYSIS': type_of_analysis,  
                'YOUTUBE_URL': youtube_url, 
                'MAIN_BUCKET_NAME': main_bucket_name,
                'NAMESPACE_NAME': namespace_name,
                'SCHEMA_NAME': schema_name}
)

job_run_watch = job_run_env.watch()

----

## **| 4. Query the Autonomous Database to see results**

##### When you would like to see the results in the notebook session, you can query the results here. Add your credentials below and run the cell.

In [ ]:
user_name = ""                         ## User name of the database, same as schema name. Example "ocw"
password = ""                          ## Password. Example "password12345"
service_name = ""                      ## Service Name, including _high, _medium, or _low. Example: db202112101358_high. Make sure you use lower case letters. Example = "db202112101358_high"
full_wallet_name = ""                  ## Please define your full wallet name, including "Wallet_" and including ".zip". Example: "Wallet_DB202112101358.zip"

In [ ]:
import pandas as pd
import ads

creds = {
    "user_name": user_name,
    "password":  password,
    "service_name": service_name,
    "wallet_location": f"/home/datascience/ocw_las_vegas/job_artifacts/wallet/{full_wallet_name}"             ## Different in Job versus in Notebook
}


# simple read of a SQL query into a dataframe with no bind variables
df = pd.DataFrame.ads.read_sql("SELECT * FROM ocw_run_results",connection_parameters=creds)
df.head()

# **The End**